In [196]:
from itertools import chain
import jax.numpy as jnp
from jax import jit, vmap, grad
from jax import random

from membrane_simulation import obtain_qoi_plane, plot_membrane_shape, simulate_ringed_membrane, plot_ode_sol, W_grads, shoot_x
from membrane_simulation import _MU_elastic, _MU_stiff, _JM_elastic, _JM_stiff

In [197]:
import pickle
dict_name = 'model_data_dictionary_N_Pa.pkl'
data_dict = pickle.load(open(dict_name, 'rb'))
new_dict = data_dict
keys=list(new_dict.keys())
N_to_lbf = 0.224809
lbf_to_N = 1/N_to_lbf
PSI_to_Pa = 6894.76
Pa_to_PSI = 1/PSI_to_Pa
import numpy as np

# ...existing code...

# Create flattened pressure and force variables
locals().update({f"pressure{i+1}": np.array(new_dict[key][0][1]).flatten() for i, key in enumerate(keys)})
locals().update({f"force{i+1}": np.array(new_dict[key][1]).flatten() for i, key in enumerate(keys)})
locals().update({f"height{i+1}": [x[0] for x in new_dict[key][0][0]] for i, key in enumerate(keys)})

# Then create plane tuples
for i in range(len(keys)):
    locals()[f"plane{i+1}"] = (
        keys[i],
        (locals()[f"pressure{i+1}"], locals()[f"force{i+1}"], locals()[f"height{i+1}"])
    )

In [200]:
# Create tuples with nested structure, filtering out nan values
locals().update({
    f"plane_values{i+1}": (
        float(tuple(map(float, keys[i].strip('()[]').replace(' ', '').split(',')))[0]),
        # Convert single-element tuple to float if length is 1
        float(next(iter(filtered))) if len(filtered := tuple(x for x in tuple(map(float, keys[i].strip('()[]').replace(' ', '').split(',')))[1:] if str(x) != 'nan')) == 1
        else filtered
    )
    for i in range(len(keys))
})

plane_values_dict = {}
for i, key in enumerate(keys):
    values = tuple(map(float, key.strip('()[]').replace(' ', '').split(',')))
    filtered = tuple(x for x in values[1:] if str(x) != 'nan')
    
    plane_values_dict[f"plane_values{i+1}"] = (
        float(values[0]),
        float(filtered[0]) if len(filtered) == 1 else filtered
    )

# Update locals with plane_values
locals().update(plane_values_dict)

# Now create membrane variables with converted values and index
membrane_dict = {}
for i in range(len(keys)):
    plane_value = plane_values_dict[f"plane_values{i+1}"]
    inner_values = plane_value[1] if isinstance(plane_value[1], tuple) else (plane_value[1],)
    membrane_dict[f"membrane{i+1}"] = (
        0.001 * plane_value[0],
        tuple(0.001 * x for x in inner_values) + (0.07,),
        i + 1  # Add the index number
    )

# Update locals with membrane variables
locals().update(membrane_dict)



In [206]:
# Define parameters for each plane, taken from experimental data

ringless_membranes = [membrane1, membrane2, membrane3,
                      membrane4, membrane5, membrane6
]

# Initialize list to store simulation planes
sim_planes = []

# Generate all planes in a loop
for i, (thickness, radii, force_idx) in enumerate(ringless_membranes, 1):
    FF1, PP1, heights11, best_x1 = obtain_qoi_plane(
        thickness,
        radii,
        ('elast',),
        F_min=0,
        F_max=max(globals()[f'force{force_idx}']),
        p_min=40,
        p_max=max(globals()[f'pressure{force_idx}']),
        dx_init=0.012,
        init_guess=5.9,
        max_iter=460,
        num_ps=110,
        num_Fs=110,
        atol=4e-3,
        return_best_x=True
    )
    sim_planes.append((keys[i],(FF1, PP1, heights11)))
    globals()[f'sim_plane{i}'] = (FF1, PP1, heights11)



100%|██████████| 460/460 [05:39<00:00,  1.35it/s]

Loop finished after 460 iterations, but tolerance has not been reached yet for some points.
Overall, 6749 pressure/force pairs were successfull.
Could not find roots for the remaining 5351.

Computing heights for successfull pressure/force pairs...


Found 4403 heights larger than 1mm. Other 2346 were less than that.


100%|██████████| 460/460 [04:27<00:00,  1.72it/s]

Loop finished after 460 iterations, but tolerance has not been reached yet for some points.
Overall, 6838 pressure/force pairs were successfull.
Could not find roots for the remaining 5262.

Computing heights for successfull pressure/force pairs...


...

Loop finished after 460 iterations, but tolerance has not been reached yet for some points.
Overall, 6644 pressure/force pairs were successfull.
Could not find roots for the remaining 5456.

Computing heights for successfull pressure/force pairs...


Found 4122 heights larger than 1mm. Other 2522 were less than that.


100%|██████████| 460/460 [04:27<00:00,  1.72it/s]

Loop finished after 460 iterations, but tolerance has not been reached yet for some points.
Overall, 6924 pressure/force pairs were successfull.
Could not find roots for the remaining 5176.

Computing heights for successfull pressure/force pairs...


Found 4464 heights larger than 1mm. Other 2460 were less than that.


In [ ]:
# # Uncomment this to save the ringless membranes results
# import pickle
# with open('ringless_membranes_sim.pkl', 'wb') as f:
#     pickle.dump(sim_planes, f)


(0.001, (0.0254, 0.0412, 0.0128, 0.062, 0.005, 0.07), 7)


In [278]:
all_membranes=[membrane1, membrane2, membrane3, membrane4, membrane5, membrane6, membrane7, membrane8, membrane9, membrane10, membrane11, membrane12, membrane13, membrane14, membrane15,
                  membrane16, membrane17, membrane18, membrane19, membrane20, membrane21, membrane22]

In [ ]:
# membrane 7, simply change n for the other membranes
n=7
FF1, PP1, heights11, best_x1 = obtain_qoi_plane(
        all_membranes[n-1][0],
        (all_membranes[n-1][1][0], all_membranes[n-1][1][1]-all_membranes[n-1][1][2], 
        all_membranes[n-1][1][1]+all_membranes[n-1][1][2], all_membranes[n-1][1][3]-all_membranes[n-1][1][4], 
        all_membranes[n-1][1][3]+all_membranes[n-1][1][4],all_membranes[n-1][1][5]),
        ('elast','stiff','elast','stiff','elast'),
        F_min=0,
        F_max=ceil(max(globals()[f'force{n}'])),
        p_min=40,
        p_max=round_to_hundred(max(globals()[f'pressure{n}'])),
        dx_init=0.011,
        init_guess=5.5,
        max_iter=400,
        num_ps=100,
        num_Fs=100,
        atol=1e-3,
        return_best_x=True
    )
import pickle
with open('ringed_membrane_7.pkl', 'wb') as f:
    pickle.dump((keys[n-1],(FF1, PP1, heights11)), f)

100%|██████████| 400/400 [13:37<00:00,  2.04s/it]

Loop finished after 400 iterations, but tolerance has not been reached yet for some points.
Overall, 9880 pressure/force pairs were successfull.
Could not find roots for the remaining 120.

Computing heights for successfull pressure/force pairs...


Found 5379 heights larger than 1mm. Other 4501 were less than that.


In [ ]:
# membrane 8
n=8
FF1, PP1, heights11, best_x1 = obtain_qoi_plane(
        all_membranes[n-1][0],
        (all_membranes[n-1][1][0], all_membranes[n-1][1][1]-all_membranes[n-1][1][2], 
        all_membranes[n-1][1][1]+all_membranes[n-1][1][2], all_membranes[n-1][1][3]-all_membranes[n-1][1][4], 
        all_membranes[n-1][1][3]+all_membranes[n-1][1][4],all_membranes[n-1][1][5]),
        ('elast','stiff','elast','stiff','elast'),
        F_min=0,
        F_max=ceil(max(globals()[f'force{n}'])),
        p_min=40,
        p_max=round_to_hundred(max(globals()[f'pressure{n}'])),
        dx_init=0.012,
        init_guess=5.5,
        max_iter=400,
        num_ps=100,
        num_Fs=100,
        atol=1e-3,
        return_best_x=True
    )
import pickle
with open('ringed_membrane_8.pkl', 'wb') as f:
    pickle.dump((keys[n-1],(FF1, PP1, heights11)), f)

100%|██████████| 400/400 [11:55<00:00,  1.79s/it]

Loop finished after 400 iterations, but tolerance has not been reached yet for some points.
Overall, 9913 pressure/force pairs were successfull.
Could not find roots for the remaining 87.

Computing heights for successfull pressure/force pairs...


Found 4248 heights larger than 1mm. Other 5665 were less than that.


In [286]:
# membrane 9
n=9
FF1, PP1, heights11, best_x1 = obtain_qoi_plane(
        all_membranes[n-1][0],
        (all_membranes[n-1][1][0], all_membranes[n-1][1][1]-all_membranes[n-1][1][2], 
        all_membranes[n-1][1][1]+all_membranes[n-1][1][2], all_membranes[n-1][1][3]-all_membranes[n-1][1][4], 
        all_membranes[n-1][1][3]+all_membranes[n-1][1][4],all_membranes[n-1][1][5]),
        ('elast','stiff','elast','stiff','elast'),
        F_min=0,
        F_max=ceil(max(globals()[f'force{n}'])),
        p_min=40,
        p_max=round_to_hundred(max(globals()[f'pressure{n}'])),
        dx_init=0.012,
        init_guess=5.5,
        max_iter=400,
        num_ps=100,
        num_Fs=100,
        atol=1e-3,
        return_best_x=True
    )
import pickle
with open('ringed_membrane_9.pkl', 'wb') as f:
    pickle.dump((keys[n-1],(FF1, PP1, heights11)), f)

100%|██████████| 400/400 [13:06<00:00,  1.97s/it]

Loop finished after 400 iterations, but tolerance has not been reached yet for some points.


Overall, 9711 pressure/force pairs were successfull.
Could not find roots for the remaining 289.

Computing heights for successfull pressure/force pairs...
Found 4662 heights larger than 1mm. Other 5049 were less than that.


In [288]:
# membrane 10
n=10
FF1, PP1, heights11, best_x1 = obtain_qoi_plane(
        all_membranes[n-1][0],
        (all_membranes[n-1][1][0], all_membranes[n-1][1][1]-all_membranes[n-1][1][2], 
        all_membranes[n-1][1][1]+all_membranes[n-1][1][2], all_membranes[n-1][1][3]-all_membranes[n-1][1][4], 
        all_membranes[n-1][1][3]+all_membranes[n-1][1][4],all_membranes[n-1][1][5]),
        ('elast','stiff','elast','stiff','elast'),
        F_min=0,
        F_max=ceil(max(globals()[f'force{n}'])),
        p_min=40,
        p_max=round_to_hundred(max(globals()[f'pressure{n}'])),
        dx_init=0.012,
        init_guess=5.5,
        max_iter=400,
        num_ps=100,
        num_Fs=100,
        atol=1e-3,
        return_best_x=True
    )
import pickle
with open('ringed_membrane_10.pkl', 'wb') as f:
    pickle.dump((keys[n-1],(FF1, PP1, heights11)), f)

100%|██████████| 400/400 [13:32<00:00,  2.03s/it]

Loop finished after 400 iterations, but tolerance has not been reached yet for some points.


Overall, 9247 pressure/force pairs were successfull.
Could not find roots for the remaining 753.

Computing heights for successfull pressure/force pairs...
Found 5821 heights larger than 1mm. Other 3426 were less than that.


In [290]:
# membrane 11
n=11
FF1, PP1, heights11, best_x1 = obtain_qoi_plane(
        all_membranes[n-1][0],
        (all_membranes[n-1][1][0], all_membranes[n-1][1][1]-all_membranes[n-1][1][2], 
        all_membranes[n-1][1][1]+all_membranes[n-1][1][2], all_membranes[n-1][1][3]-all_membranes[n-1][1][4], 
        all_membranes[n-1][1][3]+all_membranes[n-1][1][4],all_membranes[n-1][1][5]),
        ('elast','stiff','elast','stiff','elast'),
        F_min=0,
        F_max=ceil(max(globals()[f'force{n}'])),
        p_min=40,
        p_max=round_to_hundred(max(globals()[f'pressure{n}'])),
        dx_init=0.012,
        init_guess=5.5,
        max_iter=400,
        num_ps=100,
        num_Fs=100,
        atol=1e-3,
        return_best_x=True
    )
import pickle
with open('ringed_membrane_11.pkl', 'wb') as f:
    pickle.dump((keys[n-1],(FF1, PP1, heights11)), f)

100%|██████████| 400/400 [13:58<00:00,  2.10s/it]

Loop finished after 400 iterations, but tolerance has not been reached yet for some points.


Overall, 9577 pressure/force pairs were successfull.
Could not find roots for the remaining 423.

Computing heights for successfull pressure/force pairs...
Found 5593 heights larger than 1mm. Other 3984 were less than that.


In [296]:
# membrane 14
n=14
FF1, PP1, heights11, best_x1 = obtain_qoi_plane(
        all_membranes[n-1][0],
        (all_membranes[n-1][1][0], all_membranes[n-1][1][1]-all_membranes[n-1][1][2], 
        all_membranes[n-1][1][1]+all_membranes[n-1][1][2], all_membranes[n-1][1][3]-all_membranes[n-1][1][4], 
        all_membranes[n-1][1][3]+all_membranes[n-1][1][4],all_membranes[n-1][1][5]),
        ('elast','stiff','elast','stiff','elast'),
        F_min=0,
        F_max=ceil(max(globals()[f'force{n}'])),
        p_min=40,
        p_max=round_to_hundred(max(globals()[f'pressure{n}'])),
        dx_init=0.012,
        init_guess=5.5,
        max_iter=365,
        num_ps=100,
        num_Fs=100,
        atol=1e-3,
        return_best_x=True
    )
import pickle
with open('ringed_membrane_14.pkl', 'wb') as f:
    pickle.dump((keys[n-1],(FF1, PP1, heights11)), f)

100%|██████████| 365/365 [12:23<00:00,  2.04s/it]

Loop finished after 365 iterations, but tolerance has not been reached yet for some points.


Overall, 9858 pressure/force pairs were successfull.
Could not find roots for the remaining 142.

Computing heights for successfull pressure/force pairs...
Found 6577 heights larger than 1mm. Other 3281 were less than that.


In [ ]:
# membrane 15
n=15
FF1, PP1, heights11, best_x1 = obtain_qoi_plane(
        all_membranes[n-1][0],
        (all_membranes[n-1][1][0], all_membranes[n-1][1][1]-all_membranes[n-1][1][2], 
        all_membranes[n-1][1][1]+all_membranes[n-1][1][2], all_membranes[n-1][1][3]-all_membranes[n-1][1][4], 
        all_membranes[n-1][1][3]+all_membranes[n-1][1][4],all_membranes[n-1][1][5]),
        ('elast','stiff','elast','stiff','elast'),
        F_min=0,
        F_max=ceil(max(globals()[f'force{n}'])),
        p_min=40,
        p_max=round_to_hundred(max(globals()[f'pressure{n}'])),
        dx_init=0.012,
        init_guess=5.5,
        max_iter=365,
        num_ps=100,
        num_Fs=100,
        atol=1e-3,
        return_best_x=True
    )
import pickle
with open('ringed_membrane_15.pkl', 'wb') as f:
    pickle.dump((keys[n-1],(FF1, PP1, heights11)), f)

100%|██████████| 365/365 [12:11<00:00,  2.00s/it]

Loop finished after 365 iterations, but tolerance has not been reached yet for some points.


Overall, 9459 pressure/force pairs were successfull.
Could not find roots for the remaining 541.

Computing heights for successfull pressure/force pairs...
Found 5595 heights larger than 1mm. Other 3864 were less than that.


In [304]:
# membrane 17
n=17
FF1, PP1, heights11, best_x1 = obtain_qoi_plane(
        all_membranes[n-1][0],
        (all_membranes[n-1][1][0], all_membranes[n-1][1][1]-all_membranes[n-1][1][2], 
        all_membranes[n-1][1][1]+all_membranes[n-1][1][2], all_membranes[n-1][1][3]-all_membranes[n-1][1][4], 
        all_membranes[n-1][1][3]+all_membranes[n-1][1][4],all_membranes[n-1][1][5]),
        ('elast','stiff','elast','stiff','elast'),
        F_min=0,
        F_max=ceil(max(globals()[f'force{n}'])),
        p_min=40,
        p_max=round_to_hundred(max(globals()[f'pressure{n}'])),
        dx_init=0.011,
        init_guess=5.5,
        max_iter=400,
        num_ps=100,
        num_Fs=100,
        atol=1e-3,
        return_best_x=True
    )
import pickle
with open('ringed_membrane_17.pkl', 'wb') as f:
    pickle.dump((keys[n-1],(FF1, PP1, heights11)), f)

100%|██████████| 400/400 [12:33<00:00,  1.88s/it]

Loop finished after 400 iterations, but tolerance has not been reached yet for some points.


Overall, 9731 pressure/force pairs were successfull.
Could not find roots for the remaining 269.

Computing heights for successfull pressure/force pairs...
Found 4947 heights larger than 1mm. Other 4784 were less than that.


In [306]:
# membrane 18
n=18
FF1, PP1, heights11, best_x1 = obtain_qoi_plane(
        all_membranes[n-1][0],
        (all_membranes[n-1][1][0], all_membranes[n-1][1][1]-all_membranes[n-1][1][2], 
        all_membranes[n-1][1][1]+all_membranes[n-1][1][2], all_membranes[n-1][1][3]-all_membranes[n-1][1][4], 
        all_membranes[n-1][1][3]+all_membranes[n-1][1][4],all_membranes[n-1][1][5]),
        ('elast','stiff','elast','stiff','elast'),
        F_min=0,
        F_max=ceil(max(globals()[f'force{n}'])),
        p_min=40,
        p_max=round_to_hundred(max(globals()[f'pressure{n}'])),
        dx_init=0.011,
        init_guess=5.5,
        max_iter=400,
        num_ps=100,
        num_Fs=100,
        atol=1e-3,
        return_best_x=True
    )
import pickle
with open('ringed_membrane_18.pkl', 'wb') as f:
    pickle.dump((keys[n-1],(FF1, PP1, heights11)), f)

100%|██████████| 400/400 [11:56<00:00,  1.79s/it]

Loop finished after 400 iterations, but tolerance has not been reached yet for some points.


Overall, 8765 pressure/force pairs were successfull.
Could not find roots for the remaining 1235.

Computing heights for successfull pressure/force pairs...
Found 4626 heights larger than 1mm. Other 4139 were less than that.


In [308]:
# membrane 19
n=19
FF1, PP1, heights11, best_x1 = obtain_qoi_plane(
        all_membranes[n-1][0],
        (all_membranes[n-1][1][0], all_membranes[n-1][1][1]-all_membranes[n-1][1][2], 
        all_membranes[n-1][1][1]+all_membranes[n-1][1][2], all_membranes[n-1][1][3]-all_membranes[n-1][1][4], 
        all_membranes[n-1][1][3]+all_membranes[n-1][1][4],all_membranes[n-1][1][5]),
        ('elast','stiff','elast','stiff','elast'),
        F_min=0,
        F_max=ceil(max(globals()[f'force{n}'])),
        p_min=40,
        p_max=round_to_hundred(max(globals()[f'pressure{n}'])),
        dx_init=0.011,
        init_guess=5.5,
        max_iter=400,
        num_ps=100,
        num_Fs=100,
        atol=1e-3,
        return_best_x=True
    )
import pickle
with open('ringed_membrane_19.pkl', 'wb') as f:
    pickle.dump((keys[n-1],(FF1, PP1, heights11)), f)

100%|██████████| 400/400 [13:26<00:00,  2.02s/it]

Loop finished after 400 iterations, but tolerance has not been reached yet for some points.


Overall, 9708 pressure/force pairs were successfull.
Could not find roots for the remaining 292.

Computing heights for successfull pressure/force pairs...
Found 5404 heights larger than 1mm. Other 4304 were less than that.


In [310]:
# membrane 20
n=20
FF1, PP1, heights11, best_x1 = obtain_qoi_plane(
        all_membranes[n-1][0],
        (all_membranes[n-1][1][0], all_membranes[n-1][1][1]-all_membranes[n-1][1][2], 
        all_membranes[n-1][1][1]+all_membranes[n-1][1][2], all_membranes[n-1][1][3]-all_membranes[n-1][1][4], 
        all_membranes[n-1][1][3]+all_membranes[n-1][1][4],all_membranes[n-1][1][5]),
        ('elast','stiff','elast','stiff','elast'),
        F_min=0,
        F_max=ceil(max(globals()[f'force{n}'])),
        p_min=40,
        p_max=round_to_hundred(max(globals()[f'pressure{n}'])),
        dx_init=0.011,
        init_guess=5.5,
        max_iter=400,
        num_ps=100,
        num_Fs=100,
        atol=1e-3,
        return_best_x=True
    )
import pickle
with open('ringed_membrane_20.pkl', 'wb') as f:
    pickle.dump((keys[n-1],(FF1, PP1, heights11)), f)

100%|██████████| 400/400 [12:17<00:00,  1.84s/it]

Loop finished after 400 iterations, but tolerance has not been reached yet for some points.


Overall, 9837 pressure/force pairs were successfull.
Could not find roots for the remaining 163.

Computing heights for successfull pressure/force pairs...
Found 4589 heights larger than 1mm. Other 5248 were less than that.


In [313]:
# membrane 22
n=22
FF1, PP1, heights11, best_x1 = obtain_qoi_plane(
        all_membranes[n-1][0],
        (all_membranes[n-1][1][0], all_membranes[n-1][1][1]-all_membranes[n-1][1][2], 
        all_membranes[n-1][1][1]+all_membranes[n-1][1][2], all_membranes[n-1][1][3]-all_membranes[n-1][1][4], 
        all_membranes[n-1][1][3]+all_membranes[n-1][1][4],all_membranes[n-1][1][5]),
        ('elast','stiff','elast','stiff','elast'),
        F_min=0,
        F_max=ceil(max(globals()[f'force{n}'])),
        p_min=40,
        p_max=round_to_hundred(max(globals()[f'pressure{n}'])),
        dx_init=0.011,
        init_guess=5.5,
        max_iter=400,
        num_ps=100,
        num_Fs=100,
        atol=1e-3,
        return_best_x=True
    )
import pickle
with open('ringed_membrane_22.pkl', 'wb') as f:
    pickle.dump((keys[n-1],(FF1, PP1, heights11)), f)

100%|██████████| 400/400 [13:35<00:00,  2.04s/it]

Loop finished after 400 iterations, but tolerance has not been reached yet for some points.


Overall, 9588 pressure/force pairs were successfull.
Could not find roots for the remaining 412.

Computing heights for successfull pressure/force pairs...
Found 7040 heights larger than 1mm. Other 2548 were less than that.
